In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
from yahoo_fin.stock_info import get_data
Top10CC = ['BTC-USD', 'ETH-USD', 'USDT-USD', 'XRP-USD', 'BCH-USD', 'LINK-USD','BNB-USD',  'ADA-USD', 'LTC-USD',
           'EOS-USD']
hisPrices = {}
for cc in Top10CC:
    hisPrices[cc] = get_data(cc, '1/1/2019', datetime.date.today(),True, '1wk')
    
Prices = pd.DataFrame(index=np.arange(hisPrices['BTC-USD'].shape[0]),columns = Top10CC)
Prices.set_index(hisPrices['BTC-USD'].index, inplace = True)
for x in hisPrices:
    Prices[x] = hisPrices[x].adjclose
hisPrices.clear

<function dict.clear>

In [3]:
returns = np.log(Prices/Prices.shift(1)).fillna(value=0) #log(R_t/R_t-1)

In [4]:
returns.mean()

BTC-USD     0.015092
ETH-USD     0.011888
USDT-USD   -0.000135
XRP-USD    -0.001886
BCH-USD     0.004595
LINK-USD    0.036093
BNB-USD     0.015039
ADA-USD     0.008156
LTC-USD     0.007649
EOS-USD    -0.000315
dtype: float64

In [5]:
import tushare as ts
ts.set_token('d22a3bc6484b9fd9618e97dfd6249e49d7146267e1167bdc375de7c1')
pro = ts.pro_api()
df = pro.libor(curr_type='USD', start_date='20190101', end_date='20201001')

In [6]:
def findLIBOR(date):
    for i in df.index:
        if date == df.loc[i]['date']:
            return df.loc[i]['1w']
            break
    return 0

In [7]:
LIBOR = returns.copy(deep =True).drop(columns = Top10CC)
LIBOR['LIBOR']=0.0
for i in LIBOR.index:
    d = str(i).split(" ")[0].split("-")
    date = d[0]+d[1]+d[2]
    LIBOR.loc[i]['LIBOR'] = findLIBOR(date)
# change 0.0 to the adjacent value
for i in range(1,len(LIBOR)):
    if LIBOR.iloc[i]['LIBOR'] == 0.0:
        LIBOR.iloc[i]['LIBOR'] = LIBOR.iloc[i-1]['LIBOR']

In [8]:
LIBOR

,LIBOR
2019-01-01,0.00000
2019-01-08,2.40875
2019-01-15,2.40200
2019-01-22,2.40888
2019-01-29,2.41388
...,...
2020-10-27,0.10563
2020-11-03,0.10563
2020-11-10,0.10563
2020-11-17,0.10563


In [9]:
def getMean(returns,start,end):
    return returns.iloc[start:end].mean()
def getCov(returns,start,end):
    return returns.iloc[start:end].cov()

In [10]:
def getRisk_free(LIBOR,start,end):
    return LIBOR.iloc[start:end]['LIBOR'].mean()/5200

In [11]:
from functools import reduce
def std(w,cov):
    return np.sqrt(reduce(np.dot,[w,cov,w.T]))

In [12]:
np.set_printoptions(suppress=True)

In [13]:
import scipy.optimize as solver
def getOptimalPortfolio(returns, LIBOR,start,end):
    
    #general initialization
    mean = getMean(returns,start,end)
    cov = getCov(returns,start,end)
    risk_free =  getRisk_free(LIBOR,start,end)
    max_sharpe = 0.0
    weights=[]
    
    #Initialization of solver.minimize
    return_change_range = np.arange(min(mean),max(mean),(max(mean)-min(mean))/100)
    w0 = np.array([1/len(Top10CC) for x in range(len(Top10CC))]) #equal weights
    #bounds = tuple((0,1) for x in range(len(Top10CC))) #boundary of each weight. (0,1)means no short selling
    
    for i in return_change_range:
        constraints = [{'type':'eq','fun': lambda x: sum(x) - 1.0}, {'type':'eq', 'fun': lambda x: sum(x*mean) - i}]
        outcome = solver.minimize(std, args=(cov),x0=w0, constraints = constraints) #args is passed to std()
        if (i-risk_free)/outcome.fun > max_sharpe:
            max_sharpe = (i-risk_free)/outcome.fun
            weights.append(outcome.x)
            
    optimalReturn = sum(weights[-1]*mean)
    optimalWeights = weights[-1]
    
    return [max_sharpe, optimalReturn, optimalWeights]

In [14]:
getOptimalPortfolio(returns, LIBOR,0,len(returns))

[0.3796660584458224,
 0.035713152592740396,
 array([ 0.80853415,  0.40726537,  0.92148326, -1.09854703, -0.05212686,
         0.268388  ,  0.29210959,  0.04874364,  0.28819168, -0.88404179])]

In [15]:
weights = getOptimalPortfolio(returns, LIBOR,0,len(returns))[2]

In [16]:
weights

array([ 0.80853415,  0.40726537,  0.92148326, -1.09854703, -0.05212686,
        0.268388  ,  0.29210959,  0.04874364,  0.28819168, -0.88404179])

In [17]:
type(weights)

numpy.ndarray

In [22]:
np.around(weights, decimals=3)

array([ 0.809,  0.407,  0.921, -1.099, -0.052,  0.268,  0.292,  0.049,
        0.288, -0.884])

In [ ]:
for i in weights:
    print(i)